In [ ]:
# %load_ext autoreload
# %autoreload 2

import torch

from data import data_helper
from data.data_helper import available_datasets
from models import model_factory
from optimizer.optimizer_helper import get_optim_and_scheduler
from torch.nn import functional as F
from torch import nn

from utils.Logger import Logger

import torchvision
import matplotlib.pyplot as plt

from train_jigsaw import Trainer
from utils import vis

class Container():
    pass

args = Container()
args.batch_size = 128
args.n_classes = 7
args.learning_rate = 0.001
args.epochs = 30
args.network = "caffenet"
args.val_size = 0.1
args.tf_logger = True
args.folder_name = "tmp" # odd_one_out
args.jigsaw_n_classes = 31

args.train_all = True
# args.jig_weight = 0.9
# args.bias_whole_image = 0.8
args.TTA = False
args.suffix = ""
args.image_size = 222
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

source = sorted(["photo", "cartoon", "sketch", "art_painting"])
args.classify_only_sane = True
args.limit_samples = 1000

for args.jig_weight in [0.4]: 
    for args.bias_whole_image in [0.8]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()


cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 3000, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/tmp/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.4_TAll_bias0.8_classifyOnlySane_146
New epoch - lr: 0.001


In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
args.epochs = 30
args.jigsaw_n_classes = 31
for args.bias_whole_image in [None, 0.1, 0.3, 0.5]:
    for args.classify_only_sane in [True, False]:
        print("===============================\n")
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jigsaw_n_classes in [1,3,5,10,20]:
    args.epochs = int((80/95)*args.jigsaw_n_classes+15)
    for k, x in enumerate(source):
        args.source = source[:k]+source[k+1:]
        args.target = x
        for i in range(3):
            print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                              args.target, 
                                                              args.jigsaw_n_classes,
                                                              i))
            trainer = Trainer(args, device)
            trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % ("-".join(args.source),
                                                            args.target,args.epochs, args.jig_weight))

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.title("%s->%s eps:%d jigweight:%.2f" % (str(source),target,epochs, jig_weight))
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.show()

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
import numpy as np

conv1 = model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
plt.plot(memory["train"], label="train")
plt.plot(memory["val"], label="val")
plt.show()

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
from data.data_helper import get_val_dataloader, get_train_dataloader
from os.path import join, dirname
# from data.JigsawLoader import JigsawTestDataset
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision

loader = get_val_dataloader("photo",31,batch_size=10,multi=False)
# loader, _ = get_train_dataloader(["photo"],31)

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    return inp

# dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)
# test = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
iter_c = iter(loader)
(tmp, v, c), d = next(iter_c)

for x in range(tmp.shape[0]):
#     image = tmp[0, x]
    image = torchvision.utils.make_grid(tmp[x],1,normalize=True)
    plt.imshow(to_plt(image))
    plt.show()

    
# print(v.max(), v.min())

In [ ]:
tmp.max(), tmp.min()

In [ ]:
from data.JigsawLoader import JigsawDataset
from PIL import Image
import torchvision.transforms as transforms

class JigsawTestDataset(JigsawDataset):
    def __init__(self, *args, **xargs):
        super().__init__(*args, **xargs)
        self._augment_tile = transforms.Compose([
#             transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]
        img = Image.open(framename).convert('RGB')
        img = self._image_transformer(img)

        w = float(img.size[0]) / self.grid_size
        n_grids = self.grid_size ** 2
        tiles = [None] * n_grids
        for n in range(n_grids):
            y = int(n / self.grid_size)
            x = n % self.grid_size
            tile = img.crop([x * w, y * w, (x + 1) * w, (y + 1) * w])
            tile = self._augment_tile(tile)
            tiles[n] = tile

        data = torch.stack(tiles, 0)
        return self.returnFunc(data), 0, int(self.labels[index])


In [ ]:
dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)

In [ ]:
sorted([np.random.randint(9) for x in range(50)])

In [ ]:
import numpy as np

In [ ]:
perms = np.load("permutations_31.npy")
perms

In [ ]:
def stuff():
    print("done")

a = {"ciao":0}
try:
    a["cigao"]
except KeyError:
    stuff()

In [ ]:
a = list(range(10))
a[:None]